In [19]:
import pandas as pd
import numpy as np
import pickle
from scipy import stats 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import warnings
import sidetable
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from geopy.geocoders import Nominatim

plt.rcParams["figure.figsize"] = (10,8)

pd.options.display.max_columns = None


## EDA

In [20]:
df = pd.read_csv('data/encuesta_limpia.csv', index_col= 0)
df.sample()

,age,gender,Q3,Q4,Q5,Q7,Q8,Q9,Q11,Q14,Q20,Q21,Q22,Q24,Q25,Q32,Q33,Q41,q_10_completa,q_27_completa,q_29_completa,q_40_completa,q_42_completa
24795,25-29,Man,Saudi Arabia,Bachelor’s degree,Software Engineer,"Python, SQL, C, C++, Java, Javascript, Other",Other,"Visual Studio ,Visual Studio Code (VSCode)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def exploracion(df):

    print(f"El dataframe tiene {df.shape[0]} filas y {df.shape[1]} columnas")  
    print("---")
    print(df.info())
    print("---")
    columnas = [df.columns[num] for num in range(len(df.columns))]
    print(f"Los nombres de las columnas son {columnas}")
    print("---")
    print(f"A continuación se muestran los principales estadísticos numéricos del dataframe {df}:")
    print(df.describe().T)
    print(f"A continuación se muestran los principales estadísticos categóricos del dataframe {df}:")
    print(df.describe(include = "object").T)
    print("---")
    print("Los porcentajes de valores nulos por cada columna son:")
    print(df.isnull().sum() * 100/df.shape[0])
    print("---")
    valores_unicos = [len(df[col].value_counts())for col in columnas]
    valores_unicos_por_columna = list(zip(columnas, valores_unicos))
    print("Estos son los valores únicos por cada columna")
    print(valores_unicos_por_columna)
    print("---")
    print("Los valores duplicados por cada columna son:")
    print(df.duplicated().sum())


In [22]:
exploracion(df)

El dataframe tiene 25973 filas y 23 columnas
---
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25973 entries, 0 to 25972
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            25973 non-null  object
 1   gender         25973 non-null  object
 2   Q3             25973 non-null  object
 3   Q4             25973 non-null  object
 4   Q5             25973 non-null  object
 5   Q7             24941 non-null  object
 6   Q8             24940 non-null  object
 7   Q9             24934 non-null  object
 8   Q11            24720 non-null  object
 9   Q14            23970 non-null  object
 10  Q20            16325 non-null  object
 11  Q21            16251 non-null  object
 12  Q22            16126 non-null  object
 13  Q24            15635 non-null  object
 14  Q25            15391 non-null  object
 15  Q32            9484 non-null   object
 16  Q33            4656 non-null   object
 17  Q41            18489

In [23]:
df.stb.missing()

,missing,total,percent
Q33,21317,25973,82.073692
q_29_completa,19739,25973,75.998152
Q32,16489,25973,63.485158
q_27_completa,16252,25973,62.572672
Q25,10582,25973,40.742309
Q24,10338,25973,39.802872
Q22,9847,25973,37.912448
Q21,9722,25973,37.431179
Q20,9648,25973,37.146267
q_42_completa,7644,25973,29.430563


In [24]:
df['Q5'].value_counts()

Student                         6804
Data Scientist                  3616
Software Engineer               2449
Other                           2393
Data Analyst                    2301
Currently not employed          1986
Research Scientist              1538
Machine Learning Engineer       1499
Business Analyst                 968
Program/Project Manager          849
Data Engineer                    668
Product Manager                  319
Statistician                     313
DBA/Database Engineer            171
Developer Relations/Advocacy      99
Name: Q5, dtype: int64

In [25]:
df.describe(include='all').T

,count,unique,top,freq
age,25973,11,25-29,4931
gender,25973,5,Man,20598
Q3,25973,66,India,7434
Q4,25973,7,Master’s degree,10132
Q5,25973,15,Student,6804
Q7,24941,825,Python,4740
Q8,24940,13,Python,20213
Q9,24934,1330,Jupyter Notebook,2341
Q11,24720,6,A laptop,16231
Q14,23970,467,"Matplotlib ,Seaborn",5697


In [26]:
df['Q33'].unique()

array([nan, 'MySQL ', 'IBM Db2 ', 'PostgreSQL ',
       'Google Cloud Firestore ', 'Oracle Database ',
       'Microsoft SQL Server ', 'Microsoft Azure SQL Database ',
       'Snowflake ', 'MongoDB ', 'Google Cloud BigQuery ',
       'Amazon DynamoDB ', 'SQLite ', 'Amazon RDS ', 'Amazon Redshift ',
       'Google Cloud BigTable ', 'Google Cloud SQL ',
       'Microsoft Azure Cosmos DB ', 'Other', 'Amazon Aurora ',
       'Google Cloud Spanner '], dtype=object)

In [27]:
df['Q32'].unique()

array([' PostgreSQL ', nan, 'MySQL , MongoDB ', ...,
       ' MongoDB , Microsoft SQL Server , Amazon Redshift ',
       'MySQL , SQLite , MongoDB , Google Cloud BigTable ',
       ' PostgreSQL , SQLite , MongoDB , Microsoft SQL Server , Amazon Redshift '],
      dtype=object)

In [28]:
df['Q33'].value_counts()

MySQL                            1110
PostgreSQL                        694
Microsoft SQL Server              611
MongoDB                           342
Oracle Database                   316
Google Cloud BigQuery             298
SQLite                            235
Microsoft Azure SQL Database      191
Amazon Redshift                   146
Snowflake                         132
Amazon RDS                         94
Google Cloud SQL                   79
IBM Db2                            78
Amazon DynamoDB                    77
Other                              64
Google Cloud Firestore             60
Microsoft Azure Cosmos DB          54
Amazon Aurora                      34
Google Cloud BigTable              31
Google Cloud Spanner               10
Name: Q33, dtype: int64

In [29]:
df['Q32'].value_counts()

 None                                                                                                                                                                                2297
MySQL                                                                                                                                                                                 754
 PostgreSQL                                                                                                                                                                           288
 Microsoft SQL Server                                                                                                                                                                 287
MySQL , SQLite                                                                                                                                                                        177
                                                                      

In [30]:
df_copia = df.copy()

In [31]:
q32_split = pd.DataFrame(df_copia['Q32'].str.split(' , '))
q32_split


,Q32
0,[ PostgreSQL ]
1,NaN
2,"[MySQL, MongoDB ]"
3,NaN
4,[ None]
...,...
25968,"[ PostgreSQL, SQLite, MongoDB, Microsoft SQL Server, Amazon Redshift ]"
25969,NaN
25970,NaN
25971,NaN


In [32]:
q32_split.explode(['Q32'])

,Q32
0,PostgreSQL
1,NaN
2,MySQL
2,MongoDB
3,NaN
...,...
25968,Amazon Redshift
25969,NaN
25970,NaN
25971,NaN


 ## Analistas Datos en todo el DF

In [36]:
df_data_analyst = df[df['Q5'] == 'Data Analyst']
df_data_analyst.sample()

,age,gender,Q3,Q4,Q5,Q7,Q8,Q9,Q11,Q14,Q20,Q21,Q22,Q24,Q25,Q32,Q33,Q41,q_10_completa,q_27_completa,q_29_completa,q_40_completa,q_42_completa
6917,25-29,Man,China,Bachelor’s degree,Data Analyst,"Python, R, SQL, Java",Python,PyCharm,A personal computer / desktop,"Matplotlib ,Seaborn ,Ggplot / ggplot2 ,Shiny ,Geoplotlib",Accounting/Finance,"1000-9,999 employees",3-4,"Analyze and understand data to influence product or business decisions, Build prototyp...","25,000-29,999","MySQL , MongoDB",MySQL,"Basic statistical software (Microsoft Excel, Google Sheets, etc.)",Kaggle Notebooks,"Alibaba Cloud , Tencent Cloud",NaN,Kaggle Learn Courses,"Kaggle (notebooks, forums, etc), Blogs (Towards Data Science, Analytics Vidhya, etc), ..."


In [38]:
df_data_analyst.describe(include= 'all').T

,count,unique,top,freq
age,2301,11,25-29,588
gender,2301,5,Man,1759
Q3,2301,65,India,564
Q4,2301,7,Master’s degree,1027
Q5,2301,1,Data Analyst,2301
Q7,2158,213,"Python, SQL",466
Q8,2157,12,Python,1587
Q9,2155,408,Jupyter Notebook,268
Q11,2148,6,A laptop,1507
Q14,2108,191,"Matplotlib ,Seaborn",445


 ## Value Counts Q3: países

In [39]:
df['Q3'].value_counts()

India                                                   7434
United States of America                                2650
Other                                                   1270
Japan                                                    921
China                                                    814
Brazil                                                   751
Russia                                                   742
Nigeria                                                  702
United Kingdom of Great Britain and Northern Ireland     550
Pakistan                                                 530
Egypt                                                    482
Germany                                                  470
Spain                                                    454
Indonesia                                                444
Turkey                                                   416
France                                                   401
South Korea             

## EUROPA


In [40]:
lista_europeos = ['Russia', 'United Kingdom of Great Britain and Northern Ireland', 'Germany', 'Spain', 'Turkey', 'France', 'Italy', 'Poland', 'Ukraine', 'Portugal', 'Greece', 'Ireland', 'Sweden', 'Switzerland', 'Belgium', 'Czech Republic', 'Romania', 'Austria', 'Belarus', 'Denmark', 'Norway']

In [46]:
df_europa = df[df['Q3'].isin(lista_europeos)]
df_europa.sample()

,age,gender,Q3,Q4,Q5,Q7,Q8,Q9,Q11,Q14,Q20,Q21,Q22,Q24,Q25,Q32,Q33,Q41,q_10_completa,q_27_completa,q_29_completa,q_40_completa,q_42_completa
18285,40-44,Woman,France,Master’s degree,Student,"Python, SQL",Python,"Visual Studio Code (VSCode) ,Jupyter Notebook",A laptop,Matplotlib,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, Google Sheets, etc.)",Colab Notebooks,NaN,NaN,University Courses (resulting in a university degree),"Email newsletters (Data Elixir, O'Reilly Data & AI, etc), Reddit (r/machinelearning, e..."


In [42]:
df_europa.value_counts().sum() 

708

In [43]:
df_europa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4590 entries, 8 to 25970
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            4590 non-null   object
 1   gender         4590 non-null   object
 2   Q3             4590 non-null   object
 3   Q4             4590 non-null   object
 4   Q5             4590 non-null   object
 5   Q7             4477 non-null   object
 6   Q8             4477 non-null   object
 7   Q9             4477 non-null   object
 8   Q11            4451 non-null   object
 9   Q14            4360 non-null   object
 10  Q20            3388 non-null   object
 11  Q21            3377 non-null   object
 12  Q22            3359 non-null   object
 13  Q24            3262 non-null   object
 14  Q25            3209 non-null   object
 15  Q32            1986 non-null   object
 16  Q33            999 non-null    object
 17  Q41            3546 non-null   object
 18  q_10_completa  4462 non-nul

- Filtrado por Data Analyst

In [48]:
data_europa = df_europa[df_europa['Q5']== 'Data Analyst']
data_europa.describe(include= 'all').T

,count,unique,top,freq
age,358,10,25-29,81
gender,358,5,Man,276
Q3,358,21,Russia,68
Q4,358,7,Master’s degree,204
Q5,358,1,Data Analyst,358
Q7,346,68,"Python, SQL",93
Q8,346,9,Python,268
Q9,346,142,Jupyter Notebook,34
Q11,345,6,A laptop,225
Q14,340,79,"Matplotlib ,Seaborn",63


In [57]:
df_europa.describe(include='all').T

,count,unique,top,freq
age,4590,11,25-29,883
gender,4590,5,Man,3779
Q3,4590,21,Russia,742
Q4,4590,7,Master’s degree,2287
Q5,4590,15,Data Scientist,848
Q7,4477,442,Python,877
Q8,4477,13,Python,3636
Q9,4477,674,Jupyter Notebook,309
Q11,4451,6,A laptop,2517
Q14,4360,247,"Matplotlib ,Seaborn",1027


## INDIA

In [50]:
df_indo = df[df['Q3'] == 'India']
df_indo.sample()

,age,gender,Q3,Q4,Q5,Q7,Q8,Q9,Q11,Q14,Q20,Q21,Q22,Q24,Q25,Q32,Q33,Q41,q_10_completa,q_27_completa,q_29_completa,q_40_completa,q_42_completa
16686,25-29,Man,India,Bachelor’s degree,Data Scientist,Python,Python,Other,A laptop,"Matplotlib ,Seaborn",Other,0-49 employees,3-4,"Analyze and understand data to influence product or business decisions, Build and/or r...","10,000-14,999",NaN,NaN,"Local development environments (RStudio, JupyterLab, etc.)","Colab Notebooks, Kaggle Notebooks",NaN,NaN,None,"Kaggle (notebooks, forums, etc), YouTube (Kaggle YouTube, Cloud AI Adventures, etc), P..."


In [52]:
df_indo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7434 entries, 0 to 25972
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            7434 non-null   object
 1   gender         7434 non-null   object
 2   Q3             7434 non-null   object
 3   Q4             7434 non-null   object
 4   Q5             7434 non-null   object
 5   Q7             7151 non-null   object
 6   Q8             7151 non-null   object
 7   Q9             7148 non-null   object
 8   Q11            7072 non-null   object
 9   Q14            6743 non-null   object
 10  Q20            3681 non-null   object
 11  Q21            3645 non-null   object
 12  Q22            3599 non-null   object
 13  Q24            3447 non-null   object
 14  Q25            3376 non-null   object
 15  Q32            1967 non-null   object
 16  Q33            1020 non-null   object
 17  Q41            4606 non-null   object
 18  q_10_completa  7099 non-nul

In [53]:
df_indo.describe(include='all').T

,count,unique,top,freq
age,7434,11,18-21,2719
gender,7434,5,Man,5681
Q3,7434,1,India,7434
Q4,7434,7,Bachelor’s degree,4008
Q5,7434,15,Student,2845
Q7,7151,417,Python,1296
Q8,7151,13,Python,6019
Q9,7148,763,Jupyter Notebook,858
Q11,7072,6,A laptop,5417
Q14,6743,246,"Matplotlib ,Seaborn",1918


- Filtrado por Data Analyst

In [62]:
data_indo = df_indo[df_indo['Q5']== 'Data Analyst']
data_indo.describe(include= 'all').T

,count,unique,top,freq
age,564,9,22-24,164
gender,564,5,Man,420
Q3,564,1,India,564
Q4,564,7,Bachelor’s degree,271
Q5,564,1,Data Analyst,564
Q7,516,103,"Python, SQL",120
Q8,516,9,Python,407
Q9,514,161,Jupyter Notebook,91
Q11,513,6,A laptop,396
Q14,498,79,"Matplotlib ,Seaborn",137


## USA

In [51]:
df_usa = df[df['Q3'] == 'United States of America']
df_usa.sample()

,age,gender,Q3,Q4,Q5,Q7,Q8,Q9,Q11,Q14,Q20,Q21,Q22,Q24,Q25,Q32,Q33,Q41,q_10_completa,q_27_completa,q_29_completa,q_40_completa,q_42_completa
6414,30-34,Woman,United States of America,Master’s degree,Business Analyst,"Python, R, SQL",Python,Jupyter Notebook,A personal computer / desktop,None,Shipping/Transportation,0-49 employees,10-14,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [55]:
df_usa.describe(include= 'all').T

,count,unique,top,freq
age,2650,11,30-34,407
gender,2650,5,Man,1949
Q3,2650,1,United States of America,2650
Q4,2650,7,Master’s degree,1235
Q5,2650,15,Student,452
Q7,2528,363,Python,322
Q8,2527,13,Python,1897
Q9,2527,556,Jupyter Notebook,157
Q11,2510,6,A laptop,1451
Q14,2471,212,None,415


- Filtrado por Data Analyst

In [61]:
data_usa = df_usa[df_usa['Q5']== 'Data Analyst']
data_usa.describe(include= 'all').T

,count,unique,top,freq
age,258,11,30-34,52
gender,258,4,Man,171
Q3,258,1,United States of America,258
Q4,258,7,Master’s degree,143
Q5,258,1,Data Analyst,258
Q7,243,53,"Python, SQL",48
Q8,242,10,Python,146
Q9,242,115,Jupyter Notebook,21
Q11,241,6,A laptop,164
Q14,235,68,None,36


## ESPAÑA

In [58]:
df_spain = df[df['Q3'] == 'Spain']
df_spain.sample()

,age,gender,Q3,Q4,Q5,Q7,Q8,Q9,Q11,Q14,Q20,Q21,Q22,Q24,Q25,Q32,Q33,Q41,q_10_completa,q_27_completa,q_29_completa,q_40_completa,q_42_completa
9964,22-24,Woman,Spain,Bachelor’s degree,Business Analyst,"Python, SQL",Python,"Spyder,Jupyter Notebook",A laptop,"Seaborn ,Plotly / Plotly Express ,Ggplot / ggplot2",Accounting/Finance,0-49 employees,0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [59]:
df_spain.describe(include='all').T

,count,unique,top,freq
age,454,11,45-49,78
gender,454,4,Man,389
Q3,454,1,Spain,454
Q4,454,7,Master’s degree,217
Q5,454,15,Data Scientist,70
Q7,442,145,Python,59
Q8,442,10,Python,345
Q9,442,191,"Visual Studio Code (VSCode) ,Jupyter Notebook",24
Q11,439,6,A laptop,242
Q14,435,95,"Matplotlib ,Seaborn",89


- Filtrado por Data Analyst

In [60]:
data_spain = df_spain[df_spain['Q5']== 'Data Analyst']
data_spain.describe(include= 'all').T

,count,unique,top,freq
age,28,8,45-49,7
gender,28,2,Man,21
Q3,28,1,Spain,28
Q4,28,5,Master’s degree,13
Q5,28,1,Data Analyst,28
Q7,28,21,"Python, SQL",4
Q8,28,6,Python,18
Q9,28,21,Jupyter Notebook,4
Q11,28,4,A laptop,18
Q14,28,18,"Matplotlib ,Seaborn",5


- Filtrados por Data Analyst hechos:

 -España

 -Europa

 -India

 -Estados Unidos

- Para el lunes:

 -